In [93]:
!pip install cufflinks


In [106]:
!pip install textblob
!pip install --upgrade plotly

!pip install wordcloud
!pip install vaderSentiment


In [2]:


import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
cf.go_offline();
import plotly.graph_objs as go
from plotly.subplots import make_subplots


import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show");
pd.set_option("display.max_columns",None)


In [3]:
df=pd.read_csv(r"C:\Users\Arjun\OneDrive\Desktop\DO IT\coding samurai\sentimental analysis\amazon.csv")

In [4]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [5]:
df=df.sort_values("wilson_lower_bound",ascending=False)
df.drop('Unnamed: 0',inplace=True,axis=1)
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [6]:
def missing_values_analysis(df):
    na_columns_=[col for col in df.columns if df[col].isnull().sum()>0]
    n_miss=df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio=(df[na_columns_].isnull().sum() / df.shape[0]*100).sort_values(ascending=True)
    missing_df=pd.concat([n_miss,np.round(ratio,2)],axis=1,keys=['Missing Values','Ratio'])
    missing_df=pd.DataFrame(missing_df)
    return missing_df

def check_dataframe(df,head=5,tail=5):
    print("SHAPE".center(82,"~"))
    print('ROWS : {}'.format(df.shape[0]))
    print('columns:{}'.format(df.shape[1]))
    print('TYPES'.center(82,'~'))
    print(df.dtypes)
    print("".center(82,'~'))
    print(missing_values_analysis(df))
    print("DUPLICATED VALUES".center(83,'~'))
    print(df.duplicated().sum())
    print("quantiles".center(82,'~'))
    numeric_df = df.select_dtypes(include=['number'])
    print(numeric_df.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

    

check_dataframe(df)
    

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ROWS : 4915
columns:11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing Values  Ratio
reviewerName               1   0.02
reviewText                 1   0.02
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~quantiles~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                       0.00  0.05   0.50        0.95       0.99         1.00
overall 

In [7]:
def check_class(dataframe):
    unique=pd.DataFrame({'Variable':dataframe.columns,
                         'Classes':[dataframe[i].nunique()\
                         for i in dataframe.columns ]})
    unique=unique.sort_values('Classes',ascending=False)
    unique=unique.reset_index(drop=True)
    return unique

check_class(df)

,Variable,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [8]:
colors=['#B34D22','#EBE00C','#1FEB0C','#0C92EB','#EB0CD5']
def variable_summary(df,column_name):
    figure= make_subplots(rows=1,cols=2,
                          subplot_titles=('CountPlot','Percentage'),
                          specs=[[{"type": "xy"},{'type':'domain'}]])
    figure.add_trace(go.Bar( y = df[column_name].value_counts().values.tolist(),
                             x = [str(i) for i in df[column_name].value_counts().index],
                             text=df[column_name].value_counts().values.tolist(),
                             textfont=dict(size=14),
                             name=column_name,
                             textposition='auto',
                             showlegend=False,
                             marker=dict(color=colors,
                                         line=dict(color='#DBE6EC',
                                                    width=1))),
                   row=1,col=1)
    figure.add_trace(go.Pie(labels=df[column_name].value_counts().keys(),
                             values=df[column_name].value_counts().values,
                            textfont=dict(size=18),
                            textposition='auto',
                            showlegend=False,
                            name=column_name,
                            marker=dict(colors=colors)),
                  row=1, col = 2)

    figure.update_layout(title={'text':column_name,
                                 'y':0.9,
                                 'x':0.5,
                                 'xanchor':'center',
                                 'yanchor':'top'},
                          template='plotly_white')
    iplot(figure)

In [9]:
variable_summary(df,'overall')

In [10]:
df.reviewText.head()

2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [11]:
example=df.reviewText[3656]
example

'Seems to be authentic.  Great speed, correct capacity, and works flawlessly with my samsung phone and card readers.  Would buy again if needed for another device.'

In [12]:
example=re.sub("[^a-zA-Z]",' ',example)
example

'Seems to be authentic   Great speed  correct capacity  and works flawlessly with my samsung phone and card readers   Would buy again if needed for another device '

In [13]:
example=example.lower().split()
example

['seems',
 'to',
 'be',
 'authentic',
 'great',
 'speed',
 'correct',
 'capacity',
 'and',
 'works',
 'flawlessly',
 'with',
 'my',
 'samsung',
 'phone',
 'and',
 'card',
 'readers',
 'would',
 'buy',
 'again',
 'if',
 'needed',
 'for',
 'another',
 'device']

In [14]:
rt= lambda x: re.sub('[^a-zA-Z]',' ',str(x))
df['reviewText']=df['reviewText'].map(rt)
df['reviewText']=df['reviewText'].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [15]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
score = SentimentIntensityAnalyzer()  
result = score.polarity_scores("iam so happy") 
print(result)


{'neg': 0.0, 'neu': 0.334, 'pos': 0.666, 'compound': 0.6115}


In [16]:
example= df['reviewText'][2331]
print(example)

this is the fastest microsdhc i ve used   i got it for a tablet but it turned out the tablet had no microsdhc slot when it arrived   so  using it for a     mp camera   it writes very quickly and then off to the next shot   i m quite pleased with the performance of this card i think there may be cards even faster   but this suits my needs of casual photography 


In [17]:
analyzer = SentimentIntensityAnalyzer()  
result = analyzer.polarity_scores(example)  
print(result)


{'neg': 0.036, 'neu': 0.88, 'pos': 0.084, 'compound': 0.5704}


In [18]:


df[['polarity','subjectivity']]=df['reviewText'].apply(lambda Text: pd.Series (TextBlob(Text).sentiment))

for index,row in df['reviewText'].items():
    score=SentimentIntensityAnalyzer().polarity_scores(row) 

    neg=score['neg']
    neu=score['neu']
    pos=score['pos']


    if neg>pos:
        df.loc[index,'sentiment']= "Negative"

    elif pos>neg:
        df.loc[index,'sentiment']= " Positive"

    else:
        df.loc[index,'sentiment']= "neutral"

In [20]:
df.head(20)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.163859,0.562259,Positive
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.103870,0.516435,Positive
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.212251,0.505394,Positive
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.143519,0.494207,Positive
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.172332,0.511282,Positive
1835,goconfigure,5,bought from bestbuy online the day it was anno...,28-02-2014,283,60,8,68,52,0.882353,0.784651,0.386111,0.616667,Positive
3981,"R. Sutton, Jr. ""RWSynergy""",5,the last few days i have been diligently shopp...,22-10-2012,777,112,27,139,85,0.805755,0.732136,0.070466,0.445756,Positive
3807,R. Heisler,3,i bought this card to replace a lost gig in...,27-02-2013,649,22,3,25,19,0.880000,0.700442,0.194643,0.675000,Negative
4306,Stellar Eller,5,while i got this card as a deal of the day o...,06-09-2012,823,51,14,65,37,0.784615,0.670334,0.217172,0.503535,Positive
4596,"Tom Henriksen ""Doggy Diner""",1,hi i ordered two card and they arrived the nex...,22-09-2012,807,82,27,109,55,0.752294,0.663595,0.088242,0.452143,Positive


In [21]:
variable_summary(df,'sentiment')

In [23]:
import pickle

# Assuming `model` is your trained model
model = ...

# Specify the filename
filename = 'sentimental_analysis.pkl'

# Save the model
with open(filename, 'wb') as file:
    pickle.dump(model, file)

print(f"Model saved to {filename}")

Model saved to sentimental_analysis.pkl
